In [3]:
source("~/software/notebook_assist/packages.R")
source("~/software/notebook_assist/functions.R")

setwd("/projects/CARDIPS/analysis/epigenome_resource")
library(readxl)

set.seed(5366)

In [4]:
### Read in GWAS coloc results from Supplemental table 5
table_s5 = read_xlsx("~/projects/Resource/RevisionTables/TableS5.xlsx",sheet = 2) 

In [66]:
table_s5$chrom = paste0("chr",str_extract(table_s5$GWAS_Index,"^[0-9]+"))
table_s5$start = as.numeric(gsub("_","",str_extract(table_s5$GWAS_Index,"_[0-9]+_"))) - 1
table_s5$end = as.numeric(gsub("_","",str_extract(table_s5$GWAS_Index,"_[0-9]+_")))
table_s5$trait_index = paste(table_s5$Trait_Description, table_s5$GWAS_Index)

table_s5_collapse = as.data.frame(rbindlist(lapply(unique(table_s5$trait_index), function(ti){
    out = table_s5[ table_s5$trait_index == ti, ]
    out$qtl_collapse = paste(unique(sort(unlist(strsplit(unique(out$QTL_Combo),split = "-")))), collapse=",")
    return(out)
})))

gwas_index_bed = unique(table_s5_collapse %>% 
                        select(chrom, start, end, trait_index,qtl_collapse,Colocalized)) %>% 
                            arrange(chrom,start,end)
index_file = "/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/gwas_qtl_distance/gwas_index2.bed"
fwrite(gwas_index_bed, index_file, sep="\t",quote=F,row.names = F,col.names = F)

In [67]:
unique(table_s5_collapse$qtl_collapse)

[1] "caQTL"            "caQTL,eQTL"       "caQTL,haQTL"      "eQTL"            
[5] "haQTL"            "eQTL,haQTL"       "caQTL,eQTL,haQTL" "NA"

In [68]:
gencode = fread("/reference/private/Gencode.v44lift38/gene_info.txt",sep="\t",data.table=F) %>% 
            filter(gene_type == "protein_coding") %>% 
            mutate(tss_start = ifelse(strand == "+", start - 1, end ),
                  tss_end = ifelse(strand == "+", start, end + 1 )) %>%
            select(chrom, tss_start, tss_end, gene_id, gene_name) %>% 
            arrange(chrom, tss_start, tss_end)


tss_file = "/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/gwas_coloc2/gwas_qtl_distance/tss.bed"
fwrite(gencode, tss_file, sep="\t",quote=F,row.names = F,col.names = F)
bedtools="/frazer01/software/bedtools-2.25.0/bin/bedtools"

cmd = paste(bedtools, "closest -a", index_file, "-b",tss_file, "-d")
gwas_closest = unique(fread(cmd = cmd, data.table =F) %>% select(V4, V5,V6,V12))
colnames(gwas_closest) = c("trait_index","qtl_collapse","coloced","distance")
fwrite(gwas_closest,"~/projects/Resource/iPSCORE_Multi-QTL_Resource/SourceData/SOURCEDATA.FIGURE5CD.txt",sep="\t",quote=F,row.names=F)